In [1]:
from pathlib import Path
from collections import Counter
import os
import re
import random
import subprocess

import anndata
import scanpy as sc
import pandas as pd
import numpy as np
import scipy

import mgitools.os_helpers as os_helpers

In [2]:
# !conda install -y scanpy
# !pip install git+https://github.com/estorrs/mgitools

In [3]:
# !pip install git+https://github.com/estorrs/mgitools
# !pip install tensorflow==2.1.0

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

In [6]:
# !pip install -e /home/estorrs/pollock/
import pollock
from pollock.models.model import PollockDataset, PollockModel, load_from_directory, predict_from_anndata

In [7]:
# !conda install -y scanpy

In [8]:
# !pip install git+https://github.com/estorrs/mgitools

In [2]:
CELL_TYPE_KEY = 'cell_type'
N_PER_CELL_TYPE = 500
DATA_DIR = '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/'
RESULTS_DIR = '/home/estorrs/pollock/benchmarking/results/01272021_harmonized_v2/'
RESULTS_CROSS_DISEASE_DIR = '/home/estorrs/pollock/benchmarking/results/01272021_teir1_cross_disease_v2'
# RESULTS_CROSS_DTYPE_DIR = '/home/estorrs/pollock/benchmarking/results/11302020_teir1_cross_datatype'
SANDBOX_DIR = '/home/estorrs/pollock/benchmarking/sandbox'

Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
Path(RESULTS_CROSS_DISEASE_DIR).mkdir(parents=True, exist_ok=True)
# Path(RESULTS_CROSS_DTYPE_DIR).mkdir(parents=True, exist_ok=True)

##### create training and validation datasets

only run if you haven't created these datasets yet

In [12]:
def cap_list(ls, n=100, split=.8, oversample=True):
    """
    Grabs items from a pool.
    
    if split * pool size is greater than n, then just randomly sample 80% of the pool
    otherwise sample 80% of the pool, then oversample so you end up with a final size of n
    """
    # just return list if it is of length 1
    if len(ls) <= 1: return ls
    cap = int(len(ls) * split)
    if cap > n:
        return random.sample(ls, n)

    if oversample:
        pool = random.sample(ls, cap)
        ## oversample to
        return random.choices(pool, k=n)

    return random.sample(ls, cap)

def balancedish_training_generator(adata, cell_type_key, n_per_cell_type, oversample=True, split=.8):
    """
    Return balanced train and validation sets
    """
    cell_type_to_idxs = {}
    for cell_id, cell_type in zip(adata.obs.index, adata.obs[cell_type_key]):
        if cell_type not in cell_type_to_idxs:
            cell_type_to_idxs[cell_type] = [cell_id]
        else:
            cell_type_to_idxs[cell_type].append(cell_id)

    cell_type_to_idxs = {k:cap_list(ls, n_per_cell_type, oversample=oversample, split=split)
                         for k, ls in cell_type_to_idxs.items()}

    train_ids = np.asarray([x for ls in cell_type_to_idxs.values() for x in ls])
    train_idxs = np.arange(adata.shape[0])[np.isin(np.asarray(adata.obs.index), train_ids)]
    val_idxs = np.delete(np.arange(adata.shape[0]), train_idxs)

    train_adata = adata[train_idxs, :]
    val_adata = adata[val_idxs, :]

    return train_adata, val_adata

# def create_train_val_datasets(adata, cell_type_key, oversample=True):
#     counts = Counter(adata.obs[cell_type_key])
#     min_count = counts.most_common()[-1][1]
#     n_per_cell_type = max(min_count, )
#     train_adata, val_adata = balancedish_training_generator(adata, cell_type_key,
#                                                             n_per_cell_type, oversample=oversample)
#     return train_adata, val_adata

In [13]:
fps = sorted(os_helpers.listfiles(DATA_DIR, regex='.h5ad$'))
fp_map = {fp.split('/')[-2]:{} for fp in fps}
for fp in fps:
    if '/_train.h5ad' not in fp and '/_val.h5ad' not in fp:
        dtype = fp.split('/')[-2]
        disease = fp.split('/')[-1].replace('.h5ad', '')
        fp_map[dtype][disease] = fp
fp_map

{'scRNAseq': {'brca': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/brca.h5ad',
  'cesc': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/cesc.h5ad',
  'hnscc': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/hnscc.h5ad',
  'melanoma': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/melanoma.h5ad',
  'myeloma': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/myeloma.h5ad',
  'pdac': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/scRNAseq/pdac.h5ad'},
 'snATACseq': {'brca': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/snATACseq/brca.h5ad',
  'brca_gene_activity': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/snATACseq/brca_gene_activity.h5ad',
  'brca_motif': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/snATACseq/brca_motif.h5ad',
  'brca_peaks': '/home/e

In [14]:
for dtype, d in fp_map.items():
    for disease, fp in d.items():
        print(dtype, disease)
        adata = sc.read_h5ad(fp)
        # check for cell type key
        if CELL_TYPE_KEY not in adata.obs: raise RuntimeError(f'{CELL_TYPE_KEY} not in {fp}')
        
        train_adata, val_adata = balancedish_training_generator(adata, CELL_TYPE_KEY, N_PER_CELL_TYPE)
        # resample validation data to make dataset smaller while keeping rare cell types
        val_adata, _ = balancedish_training_generator(val_adata, CELL_TYPE_KEY, 500, oversample=False,
                                                     split=1.)
        train_adata.write_h5ad(fp.replace('.h5ad', '_train.h5ad'))
        val_adata.write_h5ad(fp.replace('.h5ad', '_val.h5ad'))
        

scRNAseq brca


/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


scRNAseq cesc
scRNAseq hnscc
scRNAseq melanoma
scRNAseq myeloma
scRNAseq pdac
snATACseq brca
snATACseq brca_gene_activity
snATACseq brca_motif
snATACseq brca_peaks
snATACseq ccrcc
snATACseq ccrcc_gene_activity
snATACseq ccrcc_motif
snATACseq ccrcc_peaks
snATACseq gbm
snATACseq gbm_gene_activity
snATACseq gbm_motif
snATACseq gbm_peaks
snRNAseq brca
snRNAseq ccrcc


/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


snRNAseq gbm


##### load in training and validation datasets

In [3]:
fps = sorted(os_helpers.listfiles(DATA_DIR, regex='.h5ad$'))
adata_map = {fp.split('/')[-2]:{} for fp in fps}
for fp in fps:
    dtype = fp.split('/')[-2]
    disease = re.sub(r'^(.*)((_train)|(_val)).h5ad$', r'\1', fp.split('/')[-1])
    if 'peaks' not in disease:
        if disease not in adata_map[dtype] and '.h5ad' not in disease: adata_map[dtype][disease] = {}
        if 'train.h5ad' in fp:
            adata_map[dtype][disease]['train'] = fp
        if 'val.h5ad' in fp:
            adata_map[dtype][disease]['val'] = fp
for dtype, d in adata_map.items():
    for disease, m in d.items():
        print(dtype, disease)

scRNAseq brca
scRNAseq cesc
scRNAseq hnscc
scRNAseq melanoma
scRNAseq myeloma
scRNAseq pdac
snATACseq brca_gene_activity
snATACseq brca_motif
snATACseq brca
snATACseq ccrcc_gene_activity
snATACseq ccrcc_motif
snATACseq ccrcc
snATACseq gbm_gene_activity
snATACseq gbm_motif
snATACseq gbm
snRNAseq brca
snRNAseq ccrcc
snRNAseq gbm


In [4]:
# remove any datasets if you have to
adata_map['snATACseq'].pop('brca')
adata_map['snATACseq'].pop('ccrcc')
adata_map['snATACseq'].pop('gbm')

{'train': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/snATACseq/gbm_train.h5ad',
 'val': '/home/estorrs/pollock/benchmarking/data/01272021_harmonized/teir_1/snATACseq/gbm_val.h5ad'}

### run workflows

In [5]:
import traceback
def run_workflow_for_datasets(adata_map, workflow, workflow_identifier, output_dir):
    for dtype, d in adata_map.items():
#         if dtype != 'snATACseq':
        for disease, m in d.items():
            # make dir if doesnt exist yet
            directory = os.path.join(output_dir, dtype, disease)
            Path(directory).mkdir(parents=True, exist_ok=True)
            train, val = sc.read_h5ad(m['train']), sc.read_h5ad(m['val'])

            print(dtype, disease, train.shape, val.shape)
            try:
                run_workflow(workflow, workflow_identifier,
                    train, val, directory)
            except:
                print(traceback.print_exc())
            
def run_workflow_for_cross_disease(adata_map, workflow, workflow_identifier, output_dir):
    for dtype, d in adata_map.items():
        for disease1, m1 in d.items():
            for disease2, m2 in d.items():
#                 if disease1 != disease2:
                # make dir if doesnt exist yet
                directory = os.path.join(output_dir, dtype, f'{disease1}_train_{disease2}_val')
                Path(directory).mkdir(parents=True, exist_ok=True)
                train, val = sc.read_h5ad(m1['train']), sc.read_h5ad(m2['val'])

                print(dtype, f'{disease1}_train_{disease2}_val', train.shape, val.shape)
                try:
                    run_workflow(workflow, workflow_identifier,
                        train, val, directory)
                except:
                   print(traceback.print_exc())
                    
                    
def run_workflow_for_cross_datatype(adata_map, workflow, workflow_identifier, output_dir):
    for dtype1, d1 in adata_map.items():
        for dtype2, d2 in adata_map.items():
            for disease1, m1 in d1.items():
                for disease2, m2 in d2.items():
                    # make dir if doesnt exist yet
                    directory = os.path.join(output_dir, f'{dtype1}_{dtype2}',
                                             f'{dtype1}_{disease1}_train_{dtype2}_{disease2}_val')
                    Path(directory).mkdir(parents=True, exist_ok=True)
                    train, val = sc.read_h5ad(m1['train']), sc.read_h5ad(m2['val'])

                    print(f'{dtype1}_{dtype2}',
                          f'{dtype1}_{disease1}_train_{dtype2}_{disease2}_val', train.shape, val.shape)
                    run_workflow(workflow, workflow_identifier,
                        train, val, directory)

def run_workflow(workflow, workflow_identifier, train, val, output_dir):
    """
    Run the workflow defined by the workflow function.
    
    workflow function takes a train adata and a val adata as inputs,
    and returns dataframe with cell_id, groundtruth, predicted, and probability columns
    """
#     try:
        # if it is pollock it needs to know where to save the module
    if workflow_identifier == 'pollock':
        df = workflow(train, val, CELL_TYPE_KEY, os.path.join(output_dir, f'{workflow_identifier}_module'))
    else:
        df = workflow(train, val, CELL_TYPE_KEY)

    df.to_csv(os.path.join(output_dir, f'{workflow_identifier}.tsv'), sep='\t', index=False, header=True)
#     except Exception as e:
#         print('failed ' + os.path.join(output_dir, f'{workflow_identifier}.tsv'))
#         print(e)

##### pollock

In [13]:
def run_pollock_workflow(train, val, cell_type_key, module_fp):
    train.obs['is_validation'] = [False] * train.shape[0]
    val.obs['is_validation'] = [True] * val.shape[0]
    print(train.shape, val.shape)
    combined = train.concatenate(val)
    
    pds = PollockDataset(combined.copy(), cell_type_key=cell_type_key,
                     dataset_type='training', validation_key='is_validation')
    
    pm = PollockModel(pds.cell_types, pds.train_adata.shape[1], alpha=.005, latent_dim=25, learning_rate=0.0005)
    
    pm.fit(pds, epochs=15)
    
    # only score validation if cell types match
    train_cells = set(train.obs[cell_type_key])
    val_cells = set(val.obs[cell_type_key])
    score_val = True if len(train_cells.intersection(val_cells)) == len(train_cells.union(val_cells)) else False
    print(score_val)
    pm.save(pds, module_fp, score_train=True, score_val=score_val)

    preds = predict_from_anndata(val.copy(), module_fp, adata_batch_size=2500)
    
    df = pd.DataFrame.from_dict({
        'cell_id': preds.index.to_list(),
        'groundtruth': val.obs.loc[preds.index][cell_type_key].to_list(),
        'predicted': preds['predicted_cell_type'],
        'probability': preds['cell_type_probability']
    })

    return df

In [14]:
run_workflow_for_datasets(adata_map, run_pollock_workflow, 'pollock', RESULTS_DIR)

scRNAseq brca (6105, 27131) (5748, 27131)
(6105, 27131) (5748, 27131)


2021-05-14 10:24:12,075 input dataset shape: (11853, 27131)
2021-05-14 10:24:12,077 possible cell types: ['B cell', 'CD4 T cell', 'CD8 T cell', 'Dendritic', 'Endothelial', 'Erythrocyte', 'Fibroblast', 'Malignant', 'Mast', 'Monocyte', 'NK', 'Plasma', 'Treg']
2021-05-14 10:24:12,078 using validation key
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future versio

2021-05-14 10:28:15,575 epoch: 13, train loss: 33.786991119384766, val loss: 41.217529296875
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 10:28:33,645 epoch: 14, train loss: 33.698116302490234, val loss: 41.27376937866211
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev

True


/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/numpy/lib/format.py:350: UserWarning: metadata on a dtype may be saved or ignored, but will raise if saved when read. Use another form of storage.
  d['descr'] = dtype_to_descr(array.dtype)
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 10:29:06,339 0 genes in training set are missing from prediction set
2021-05-14 10:29:06,884 starting batch 1 of 3
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 10:29:06,991 0 genes in training set are missing from prediction set
2021-05-14 10:29:13,

...

snRNAseq gbm (3722, 29748) (3577, 29748)
(3722, 29748) (3577, 29748)


2021-05-14 11:09:27,060 input dataset shape: (7299, 29748)
2021-05-14 11:09:27,064 possible cell types: ['B cell', 'Endothelial', 'Fibroblast', 'Malignant', 'Microglia', 'Monocyte', 'Neuron', 'Oligodendrocytes', 'T cells']
2021-05-14 11:09:27,065 using validation key
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instea

2021-05-14 11:12:09,704 epoch: 13, train loss: 43.34651565551758, val loss: 43.93813705444336
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 11:12:22,062 epoch: 14, train loss: 43.00531768798828, val loss: 43.9701042175293
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/estorrs/miniconda3/envs/pollock_dev/

True


/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/numpy/lib/format.py:350: UserWarning: metadata on a dtype may be saved or ignored, but will raise if saved when read. Use another form of storage.
  d['descr'] = dtype_to_descr(array.dtype)
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 11:12:43,148 0 genes in training set are missing from prediction set
2021-05-14 11:12:43,681 starting batch 1 of 2
/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
2021-05-14 11:12:43,785 0 genes in training set are missing from prediction set
2021-05-14 11:12:53,

In [27]:
run_workflow_for_cross_disease(adata_map, run_pollock_workflow, 'pollock', RESULTS_CROSS_DISEASE_DIR)

scRNAseq brca_train_brca_val (6080, 27131) (5773, 27131)


2021-03-29 16:43:36,039 input dataset shape: (11853, 27131)
2021-03-29 16:43:36,041 possible cell types: ['B cell', 'CD4 T cell', 'CD8 T cell', 'Dendritic', 'Endothelial', 'Erythrocyte', 'Fibroblast', 'Malignant', 'Mast', 'Monocyte', 'NK', 'Plasma', 'Treg']
2021-03-29 16:43:36,042 using validation key
2021-03-29 16:43:54,621 epoch: 1, train loss: 38.09257888793945, val loss: 45.976531982421875
2021-03-29 16:44:10,634 epoch: 2, train loss: 37.1802978515625, val loss: 44.32764434814453
2021-03-29 16:44:26,617 epoch: 3, train loss: 36.530113220214844, val loss: 43.4514274597168
2021-03-29 16:44:42,776 epoch: 4, train loss: 36.08720397949219, val loss: 42.96310043334961
2021-03-29 16:44:58,743 epoch: 5, train loss: 35.840293884277344, val loss: 42.747711181640625
2021-03-29 16:45:14,678 epoch: 6, train loss: 35.62464141845703, val loss: 42.554988861083984
2021-03-29 16:45:30,603 epoch: 7, train loss: 35.42350769042969, val loss: 42.33623504638672
2021-03-29 16:45:46,562 epoch: 8, train los

True


2021-03-29 16:47:51,135 0 genes in training set are missing from prediction set
2021-03-29 16:47:51,782 starting batch 1 of 3
2021-03-29 16:47:51,885 0 genes in training set are missing from prediction set
2021-03-29 16:47:57,503 starting batch 2 of 3
2021-03-29 16:47:57,640 0 genes in training set are missing from prediction set
2021-03-29 16:48:03,485 starting batch 3 of 3
2021-03-29 16:48:03,605 0 genes in training set are missing from prediction set


scRNAseq brca_train_cesc_val (6080, 27131) (4277, 22928)


...

2021-03-29 20:10:04,883 0 genes in training set are missing from prediction set
2021-03-29 20:10:05,586 starting batch 1 of 2
2021-03-29 20:10:05,706 0 genes in training set are missing from prediction set
2021-03-29 20:10:12,586 starting batch 2 of 2
2021-03-29 20:10:12,659 0 genes in training set are missing from prediction set


snRNAseq gbm_train_gbm_val (3745, 29748) (3554, 29748)


2021-03-29 20:10:27,864 input dataset shape: (7299, 29748)
2021-03-29 20:10:27,865 possible cell types: ['B cell', 'Endothelial', 'Fibroblast', 'Malignant', 'Microglia', 'Monocyte', 'Neuron', 'Oligodendrocytes', 'T cells']
2021-03-29 20:10:27,866 using validation key
2021-03-29 20:10:40,863 epoch: 1, train loss: 48.398319244384766, val loss: 47.211524963378906
2021-03-29 20:10:51,454 epoch: 2, train loss: 47.63067626953125, val loss: 46.73516082763672
2021-03-29 20:11:01,980 epoch: 3, train loss: 46.666664123535156, val loss: 46.19440841674805
2021-03-29 20:11:12,114 epoch: 4, train loss: 46.169090270996094, val loss: 45.935325622558594
2021-03-29 20:11:22,591 epoch: 5, train loss: 45.88922119140625, val loss: 45.82231140136719
2021-03-29 20:11:33,000 epoch: 6, train loss: 45.67880630493164, val loss: 45.72295379638672
2021-03-29 20:11:43,645 epoch: 7, train loss: 45.442447662353516, val loss: 45.66011047363281
2021-03-29 20:11:54,174 epoch: 8, train loss: 45.214324951171875, val loss:

True


2021-03-29 20:13:16,126 0 genes in training set are missing from prediction set
2021-03-29 20:13:16,820 starting batch 1 of 2
2021-03-29 20:13:16,953 0 genes in training set are missing from prediction set
2021-03-29 20:13:24,214 starting batch 2 of 2
2021-03-29 20:13:24,307 0 genes in training set are missing from prediction set


In [ ]:
# run_workflow_for_cross_datatype(adata_map, run_pollock_workflow, 'pollock', RESULTS_CROSS_DTYPE_DIR)

###### testing stuff

In [ ]:
# a = sc.read_h5ad(adata_map['snATACseq']['gbm']['train'])
# a

In [ ]:
train, val = sc.read_h5ad(adata_map['scRNAseq']['pbmc']['train']), sc.read_h5ad(adata_map['scRNAseq']['brca']['val'])

In [ ]:
module_dir = os.path.join(SANDBOX_DIR, 'temp_module')

In [ ]:
train.obs['is_validation'] = [False] * train.shape[0]
val.obs['is_validation'] = [True] * val.shape[0]
combined = train.concatenate(val)
combined

In [ ]:
train.shape, val.shape

In [ ]:
np.count_nonzero(combined.obs['is_validation']), np.count_nonzero(~combined.obs['is_validation'])

In [ ]:
# pds = PollockDataset(train, cell_type_key=CELL_TYPE_KEY,
#                      dataset_type='training')

In [ ]:
val.shape

In [ ]:
pds = PollockDataset(combined, cell_type_key=CELL_TYPE_KEY,
                     dataset_type='training', validation_key='is_validation')

In [ ]:
pm = PollockModel(pds.cell_types, pds.train_adata.shape[1], alpha=.0001, latent_dim=25)

In [ ]:
pm.fit(pds, epochs=2)

In [ ]:
pm.save(pds, module_dir)

In [ ]:
val.shape

In [ ]:
preds = predict_from_anndata(val.copy(),
        '/home/estorrs/pollock/benchmarking/sandbox/temp_module', adata_batch_size=10000)
preds

In [ ]:
df = pd.DataFrame.from_dict({
    'cell_id': preds.index.to_list(),
    'groundtruth': val.obs.loc[preds.index][CELL_TYPE_KEY].to_list(),
    'predicted': preds['predicted_cell_type'],
    'probability': preds['cell_type_probability']
})
df

##### scanpy ingest

In [ ]:
def ingest_preprocess(adata):
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2500)
    adata.raw = adata
    adata = adata[:, adata.var.highly_variable]
    sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
    sc.pp.scale(adata)
    
    return adata

def run_scanpy_workflow(train, val, cell_type_key):
    var_names = train.var_names.intersection(val.var_names)
    train = train[:, var_names]
    val = val[:, var_names]
    
    groundtruth = val.obs[cell_type_key].to_list()

    sc.pp.pca(train)
    sc.pp.neighbors(train)
    sc.tl.umap(train)
    
    sc.tl.ingest(val, train, obs=cell_type_key)
    
    df = pd.DataFrame.from_dict({
        'cell_id': val.obs.index.to_list(),
        'groundtruth': groundtruth,
        'predicted': val.obs[cell_type_key].to_list(),
        'probability': [np.nan] * val.shape[0]
    })
    
    return df

In [ ]:
run_workflow_for_datasets(adata_map, run_scanpy_workflow, 'scanpy_ingest', RESULTS_DIR)

In [ ]:
run_workflow_for_cross_disease(adata_map, run_scanpy_workflow, 'scanpy_ingest', RESULTS_CROSS_DISEASE_DIR)

In [ ]:
# run_workflow_for_cross_datatype(adata_map, run_scanpy_workflow, 'scanpy_ingest', RESULTS_CROSS_DTYPE_DIR)

###### testing stuff

In [ ]:
train, val = adata_map['scRNAseq']['pbmc']['train'].copy(), adata_map['scRNAseq']['pbmc']['val'].copy()

In [ ]:
train, val = ingest_preprocess(train), ingest_preprocess(val)

var_names = train.var_names.intersection(val.var_names)
train = train[:, var_names]
val = val[:, var_names]

sc.pp.pca(train)
sc.pp.neighbors(train)
sc.tl.umap(train)

In [ ]:
sc.pl.umap(train, color='cell_type')

In [ ]:
sc.tl.ingest(val, train, obs=CELL_TYPE_KEY)
val.uns[f'{CELL_TYPE_KEY}_colors'] = train.uns[f'{CELL_TYPE_KEY}_colors']

In [ ]:
sc.pl.umap(val, color=[CELL_TYPE_KEY], wspace=0.5)


In [ ]:
val

In [ ]:
val.obs

##### ACTINN

In [6]:
def run_actinn_workflow(train, val, cell_type_key):
    X = train.X.toarray() if 'sparse' in str(type(train.X)) else train.X
    train_counts_df = pd.DataFrame(data=X.transpose(), index=train.var.index.to_list(),
                        columns=train.obs.index.to_list())
    X = val.X.toarray() if 'sparse' in str(type(val.X)) else val.X
    val_counts_df = pd.DataFrame(data=X.transpose(), index=val.var.index.to_list(),
                        columns=val.obs.index.to_list())
    
    train_counts_fp = os.path.join(SANDBOX_DIR, 'train_counts.txt')
    val_counts_fp = os.path.join(SANDBOX_DIR, 'val_counts.txt')
    train_counts_df.to_csv(train_counts_fp, sep='\t')
    val_counts_df.to_csv(val_counts_fp, sep='\t')
    
    train_h5_fp = os.path.join(SANDBOX_DIR, 'train.h5')
    train_annotations_fp = os.path.join(SANDBOX_DIR, 'train_annotations.txt')
    val_h5_fp = os.path.join(SANDBOX_DIR, 'val.h5')

    train.obs[[CELL_TYPE_KEY]].to_csv(train_annotations_fp, sep='\t', index=True, header=False)

    subprocess.check_output(('python', ACTINN_FORMAT, '-i', train_counts_fp,
                            '-o', train_h5_fp.replace('.h5', ''), '-f', 'txt'))
    subprocess.check_output(('python', ACTINN_FORMAT, '-i', val_counts_fp,
                            '-o', val_h5_fp.replace('.h5', ''), '-f', 'txt'))
    # dont use probablity argument or it breaks
    subprocess.check_output(('python', ACTINN_PREDICT, '-trs', train_h5_fp,
                            '-trl', train_annotations_fp, '-ts', val_h5_fp))
    
    prediction_df = pd.read_csv('predicted_label.txt', sep='\t')
    
    df = pd.DataFrame.from_dict({
        'cell_id': prediction_df['cellname'].to_list(),
        'predicted': prediction_df['celltype'].to_list(),
        'probability': [np.nan] * prediction_df.shape[0]
    })
    
    df = pd.merge(df, val.obs, left_on='cell_id', right_index=True)
    df = df[['cell_id', 'cell_type', 'predicted', 'probability']]
    df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
    
    return df
    
    
    
    


In [7]:
ACTINN_FORMAT = '/home/estorrs/ACTINN/actinn_format.py'
ACTINN_PREDICT = '/home/estorrs/ACTINN/actinn_predict.py'

run_workflow_for_datasets(adata_map, run_actinn_workflow, 'actinn', RESULTS_DIR)

scRNAseq brca (6105, 27131) (5748, 27131)
scRNAseq cesc (4661, 22928) (4276, 22928)
scRNAseq hnscc (5287, 26929) (5201, 26929)
scRNAseq melanoma (4218, 23452) (3517, 23452)


Traceback (most recent call last):
  File "<ipython-input-5-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-5-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-6-8b9bf882e9f0>", line 21, in run_actinn_workflow
    '-o', train_h5_fp.replace('.h5', ''), '-f', 'txt'))
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '('python', '/home/estorrs/ACTINN/actinn_format.py', '-i', '/home/estorrs/pollock/benchmarking/sandbox/train_counts.txt', '-o', '/home/estorrs/pollock/benchmarking/sandbox/train', '-f', 'txt')' returned non-zero exit status 1.


None
scRNAseq myeloma (3617, 24020) (3312, 24020)


Traceback (most recent call last):
  File "<ipython-input-5-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-5-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-6-8b9bf882e9f0>", line 37, in run_actinn_workflow
    df = df[['cell_id', 'cell_type', 'predicted', 'probability']]
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/site-packages/pandas/core/frame.py", line 3030, in __getitem__
    indexer = self.loc._get_listlike_indexer(key, axis=1, raise_missing=True)[1]
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/site-packages/pandas/core/indexing.py", line 1266, in _get_listlike_indexer
    self._validate_read_indexer(keyarr, indexer, axis, raise_missing=raise_missing)
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/site-packages/pandas/core/indexing.py", line 1316, in _validate_read_indexer
    raise KeyError(f"{not_found} not in index")
Key

None
scRNAseq pdac (7940, 28756) (7823, 28756)
snATACseq brca_gene_activity (3576, 19891) (3519, 19891)
snATACseq brca_motif (3576, 633) (3519, 633)


...

None
snATACseq ccrcc_gene_activity (3000, 19843) (3000, 19843)
snATACseq ccrcc_motif (3000, 633) (3000, 633)


Traceback (most recent call last):
  File "<ipython-input-5-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-5-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-6-8b9bf882e9f0>", line 26, in run_actinn_workflow
    '-trl', train_annotations_fp, '-ts', val_h5_fp))
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '('python', '/home/estorrs/ACTINN/actinn_predict.py', '-trs', '/home/estorrs/pollock/benchmarking/sandbox/train.h5', '-trl', '/home/estorrs/pollock/benchmarking/sandbox/train_annotations.txt', '-ts', '/home/estorrs/pollock/benchmarking/sandbox/val.h5')' returned non-zero exit status 1.


None
snATACseq gbm_gene_activity (3389, 19891) (2876, 19891)
snATACseq gbm_motif (3393, 633) (2872, 633)


Traceback (most recent call last):
  File "<ipython-input-5-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-5-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-6-8b9bf882e9f0>", line 26, in run_actinn_workflow
    '-trl', train_annotations_fp, '-ts', val_h5_fp))
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 411, in check_output
    **kwargs).stdout
  File "/home/estorrs/miniconda3/envs/actinn/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '('python', '/home/estorrs/ACTINN/actinn_predict.py', '-trs', '/home/estorrs/pollock/benchmarking/sandbox/train.h5', '-trl', '/home/estorrs/pollock/benchmarking/sandbox/train_annotations.txt', '-ts', '/home/estorrs/pollock/benchmarking/sandbox/val.h5')' returned non-zero exit status 1.


None
snRNAseq brca (5252, 29175) (4893, 29175)
snRNAseq ccrcc (4754, 33538) (4518, 33538)
snRNAseq gbm (3722, 29748) (3577, 29748)


In [11]:
run_workflow_for_cross_disease(adata_map, run_actinn_workflow, 'actinn', RESULTS_CROSS_DISEASE_DIR)

scRNAseq brca_train_brca_val (6080, 27131) (5773, 27131)
scRNAseq brca_train_cesc_val (6080, 27131) (4277, 22928)
scRNAseq brca_train_hnscc_val (6080, 27131) (5203, 26929)
scRNAseq brca_train_melanoma_val (6080, 27131) (3532, 23452)
failed
scRNAseq brca_train_myeloma_val (6080, 27131) (3312, 24020)
failed
scRNAseq brca_train_pbmc_val (6080, 27131) (886, 32738)
scRNAseq brca_train_pdac_val (6080, 27131) (7840, 28756)
scRNAseq cesc_train_brca_val (4660, 22928) (5773, 27131)
scRNAseq cesc_train_cesc_val (4660, 22928) (4277, 22928)
scRNAseq cesc_train_hnscc_val (4660, 22928) (5203, 26929)
scRNAseq cesc_train_melanoma_val (4660, 22928) (3532, 23452)
failed
scRNAseq cesc_train_myeloma_val (4660, 22928) (3312, 24020)
failed
scRNAseq cesc_train_pbmc_val (4660, 22928) (886, 32738)
scRNAseq cesc_train_pdac_val (4660, 22928) (7840, 28756)
scRNAseq hnscc_train_brca_val (5285, 26929) (5773, 27131)
scRNAseq hnscc_train_cesc_val (5285, 26929) (4277, 22928)
scRNAseq hnscc_train_hnscc_val (5285, 26929)

###### testing stuff

In [ ]:
train, val = adata_map['scRNAseq']['pbmc']['train'].copy(), adata_map['scRNAseq']['pbmc']['val'].copy()

In [ ]:
# train.obs['dataset'] = ['train'] * train.shape[0]
# val.obs['dataset'] = ['val'] * val.shape[0]
# combined = train.concatenate(val)
# combined

In [ ]:
train_counts_df = pd.DataFrame(data=train.X.transpose().toarray(), index=train.var.index.to_list(),
                        columns=train.obs.index.to_list())
val_counts_df = pd.DataFrame(data=val.X.transpose().toarray(), index=val.var.index.to_list(),
                        columns=val.obs.index.to_list())
train_counts_df

In [ ]:
train_counts_fp = os.path.join(SANDBOX_DIR, 'train_counts.txt')
val_counts_fp = os.path.join(SANDBOX_DIR, 'val_counts.txt')
train_counts_df.to_csv(train_counts_fp, sep='\t')
val_counts_df.to_csv(val_counts_fp, sep='\t')

python actinn_format.py -i input_file -o output_prefix -f format

python actinn_format.py -i ./test_data/train_set.txt.gz -o train_set -f txt


In [ ]:
train_h5_fp = os.path.join(SANDBOX_DIR, 'train.h5')
train_annotations_fp = os.path.join(SANDBOX_DIR, 'train_annotations.txt')
val_h5_fp = os.path.join(SANDBOX_DIR, 'val.h5')

train.obs[[CELL_TYPE_KEY]].to_csv(train_annotations_fp, sep='\t', index=True, header=False)

subprocess.check_output(('python', '/home/estorrs/ACTINN/actinn_format.py', '-i', train_counts_fp,
                        '-o', train_h5_fp.replace('.h5', ''), '-f', 'txt'))

In [ ]:
subprocess.check_output(('python', '/home/estorrs/ACTINN/actinn_format.py', '-i', val_counts_fp,
                        '-o', val_h5_fp.replace('.h5', ''), '-f', 'txt'))

In [ ]:
train.obs[[CELL_TYPE_KEY]]

python actinn_predict.py -trs training_set -trl training_label -ts test_set -lr learning_rat -ne num_epoch -ms minibatch_size -pc print_cost -op output_probability


-trs Path to the training set, must be HDF5 format with key "dge".

-trl Path to the training label (the cell types for the training set), must be tab separated text file with no column and row names.

-ts Path to test sets, must be HDF5 format with key "dge".

-lr Learning rate (default: 0.0001). We can increase the learning rate if the cost drops too slow, or decrease the learning rate if the cost drops super fast in the beginning and starts to fluctuate in later epochs.

-ne Number of epochs (default: 50). The number of epochs can be determined by looking at the cost after each epoch. If the cost starts to decrease very slowly after ceartain epoch, then the "ne" parameter should be set to that epoch number.

-ms Minibatch size (default: 128). This parameter can be set larger when training a large dataset.

-pc Print cost (default: True). Whether to print cost after each 5 epochs.

-op Output probabilities for each cell being the cell types in the training data (default: False).


In [ ]:
subprocess.check_output(('python', '/home/estorrs/ACTINN/actinn_predict.py', '-trs', train_h5_fp,
                        '-trl', train_annotations_fp, '-ts', val_h5_fp))

In [ ]:
' '.join(('python', '/home/estorrs/ACTINN/actinn_predict.py', '-trs', train_h5_fp,
                        '-trl', train_annotations_fp, '-ts', val_h5_fp,
                        '-op', 'True'))

In [ ]:
prediction_df = pd.read_csv('predicted_label.txt', sep='\t')
prediction_df

In [ ]:
df = pd.DataFrame.from_dict({
        'cell_id': prediction_df['cellname'].to_list(),
        'prediction': prediction_df['celltype'].to_list(),
        'probability': [np.nan] * val.shape[0]
    })
df

In [ ]:
val.obs

In [ ]:
df = pd.merge(df, val.obs, left_on='cell_id', right_index=True)
df = df[['cell_id', 'cell_type', 'prediction', 'probability']]
df.columns = ['cell_id', 'groundtruth', 'prediction', 'probability']
df


##### Seurat

In [ ]:
def run_seurat_transfer(train, val, cell_type_key):
    # save the input data for the seurat script
    train_counts_fp, val_counts_fp = (os.path.join(SANDBOX_DIR, 'train_counts.txt'),
                                        os.path.join(SANDBOX_DIR, 'val_counts.txt'))
    train_annotations_fp, val_annotations_fp = (os.path.join(SANDBOX_DIR, 'train_annotations.txt'),
                                                os.path.join(SANDBOX_DIR, 'val_annotations.txt'))

    ## prepare train and val count matrices
    X = train.X.toarray() if 'sparse' in str(type(train.X)) else train.X
    train_counts = pd.DataFrame(data=X.transpose().astype(np.int32), index=train.var.index,
                                columns=train.obs.index)
    train_counts.index.name = ''
    # for some reason SCTransform fails if the integer values are too high, so capping them here
    cap = pow(2, 14)
    train_counts.values[train_counts.values>cap] = cap
    train_counts.to_csv(train_counts_fp, sep='\t', header=True, index=True)
    
    X = val.X.toarray() if 'sparse' in str(type(val.X)) else val.X
    val_counts = pd.DataFrame(data=X.transpose().astype(np.int32), index=val.var.index,
                                columns=val.obs.index)
    val_counts.index.name = ''
    val_counts.values[val_counts.values>cap] = cap
    val_counts.to_csv(val_counts_fp, sep='\t', header=True, index=True)

    train.obs[[CELL_TYPE_KEY]].to_csv(train_annotations_fp, sep='\t', header=False, index=False)
    val.obs[[CELL_TYPE_KEY]].to_csv(val_annotations_fp, sep='\t', header=False, index=False)
    
    # actually run the script and read the results back in
    prediction_fp = os.path.join(SANDBOX_DIR, 'seurat_predictions.txt')
    try:
        subprocess.check_output(('Rscript', SEURAT_SCRIPT, train_counts_fp, train_annotations_fp,
                            val_counts_fp, val_annotations_fp, prediction_fp))
    except subprocess.CalledProcessError as e:
        print(f'called process error', e)
        return pd.DataFrame()
    
    # format the predictions dataframe
    df = pd.read_csv(prediction_fp, sep='\t')
    df.index = [x.replace('.', '-') for x in df.index]
    # also remove that weird X thing seurat sometimes puts there if first char is _
    df.index = [x[1:] if x[:2]=='X_' else x for x in df.index]
    df = pd.merge(df, val.obs, left_index=True, right_index=True)
    df['cell_id'] = df.index.to_list()
    try:
        df = df[['cell_id', 'cell_type', 'predicted.id', 'prediction.score.max']]        
        df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
        return df
    except KeyError as e:
        print(f'key error', e)
        return pd.DataFrame()

In [ ]:
SEURAT_SCRIPT = '/home/estorrs/pollock/benchmarking/tools/run_seurat_workflow.R'
run_workflow_for_datasets(adata_map, run_seurat_transfer, 'seurat_transfer', RESULTS_DIR)

In [ ]:
run_workflow_for_cross_disease(adata_map, run_seurat_transfer, 'seurat_transfer', RESULTS_CROSS_DISEASE_DIR)

In [ ]:
# run_workflow_for_cross_datatype(adata_map, run_seurat_transfer, 'seurat_transfer', RESULTS_CROSS_DTYPE_DIR)

###### testing stuff

In [ ]:
# train, val = adata_map['scRNAseq']['pbmc']['train'].copy(), adata_map['scRNAseq']['pbmc']['val'].copy()
train, val = sc.read_h5ad(adata_map['scRNAseq']['brca']['train']), sc.read_h5ad(adata_map['scRNAseq']['brca']['val'])

In [ ]:
pow(2, 14)

In [ ]:
# save the input data for the seurat script
train_counts_fp, val_counts_fp = (os.path.join(SANDBOX_DIR, 'train_counts.txt'),
                                    os.path.join(SANDBOX_DIR, 'val_counts.txt'))
train_annotations_fp, val_annotations_fp = (os.path.join(SANDBOX_DIR, 'train_annotations.txt'),
                                            os.path.join(SANDBOX_DIR, 'val_annotations.txt'))

## prepare train and val count matrices
X = train.X.toarray() if 'sparse' in str(type(train.X)) else train.X
train_counts = pd.DataFrame(data=X.transpose().astype(np.int32), index=train.var.index,
                            columns=train.obs.index)
train_counts.index.name = ''
# for some reason SCTransform fails if the integer values are too high, so capping them here
cap = pow(2, 14)
train_counts.values[train_counts.values>cap] = cap
train_counts.to_csv(train_counts_fp, sep='\t', header=True, index=True)

X = val.X.toarray() if 'sparse' in str(type(val.X)) else val.X
val_counts = pd.DataFrame(data=X.transpose().astype(np.int32), index=val.var.index,
                            columns=val.obs.index)
val_counts.index.name = ''
val_counts.values[val_counts.values>cap] = cap
val_counts.to_csv(val_counts_fp, sep='\t', header=True, index=True)

train.obs[[CELL_TYPE_KEY]].to_csv(train_annotations_fp, sep='\t', header=False, index=False)
val.obs[[CELL_TYPE_KEY]].to_csv(val_annotations_fp, sep='\t', header=False, index=False)

In [ ]:
train_counts

In [ ]:
train_counts

In [ ]:
type(train_counts.values), type(train_counts.values[0, 0])

In [ ]:
vals = sorted(set(train_counts.values.flatten()))
vals

In [ ]:
vals[:10], vals[-10:]

In [ ]:
train_counts.values[train_counts.values>1000] = 1000

In [ ]:
np.where(train_counts>1)

In [ ]:
# actually run the script and read the results back in
prediction_fp = os.path.join(SANDBOX_DIR, 'seurat_predictions.txt')
subprocess.check_output(('Rscript', SEURAT_SCRIPT, train_counts_fp, train_annotations_fp,
                    val_counts_fp, val_annotations_fp, prediction_fp))

In [ ]:
# format the predictions dataframe
df = pd.read_csv(prediction_fp, sep='\t')
df.index = [x.replace('.', '-') for x in df.index]
# also remove that weird X thing seurat sometimes puts there
df.index = [x[1:] if x[:2]=='X_' else x for x in df.index]
df = pd.merge(df, val.obs, left_index=True, right_index=True)
df['cell_id'] = df.index.to_list()
df = df[['cell_id', 'cell_type', 'predicted.id', 'prediction.score.max']]        
df.columns = ['cell_id', 'groundtruth', 'predicted', 'probability']
df

In [ ]:
val.obs

##### SingleCellNet

In [ ]:
# !pip install git+https://github.com/pcahan1/PySingleCellNet/

In [15]:
import pySingleCellNet as pySCN

In [16]:
import scipy

In [17]:
def run_SingleCellNet(train, val, cell_type_key):
#     if 'sparse' in str(type(train.X)): train.X = train.X.toarray()
    if 'sparse' not in str(type(train.X)): train.X = scipy.sparse.csr_matrix(train.X)
    if 'sparse' in str(type(val.X)): val.X = val.X.toarray()
    
    # save the input data for the seurat script
    cgenesA, xpairs, tspRF = pySCN.scn_train(train,
            nTopGenes=100, nRand=100, nTrees=1000, nTopGenePairs=100,
            dLevel=cell_type_key, stratify=True, limitToHVG=True, )
    predictions = pySCN.scn_classify(val, cgenesA, xpairs, tspRF, nrand = 0)
    
    df = pd.merge(predictions.obs[['SCN_class']], val.obs, left_index=True, right_index=True)
    
    df = df[['cell_type', 'SCN_class']]
    df.columns = ['groundtruth', 'predicted']
    df['cell_id'] = df.index.to_list()
    df['probability'] = [np.nan] * df.shape[0]
    df = df[['cell_id', 'groundtruth', 'predicted', 'probability']]
    

    return df

In [18]:
run_workflow_for_datasets(adata_map, run_SingleCellNet, 'SingleCellNet', RESULTS_DIR)

scRNAseq brca (6105, 27131) (5748, 27131)
HVG
Matrix normalized
There are  1061  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Endothelial
Erythrocyte
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1265 top gene pairs

Finished pair transforming the data

scRNAseq cesc (4661, 22928) (4276, 22928)
HVG
Matrix normalized
There are  984  classification genes

CD4 T cell
CD8 T cell
Endothelial
Epithelial
Erythrocyte
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
There are 1075 top gene pairs

Finished pair transforming the data

scRNAseq hnscc (5287, 26929) (5201, 26929)
HVG
Matrix normalized
There are  1077  classification genes

B cell
CD4 T cell
CD8 T cell
Endothelial
Erythrocyte
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1067 top gene pairs

Finished pair transforming the data

scRNAseq melanoma (4218, 23452) (3517, 23452)
HVG
Matrix normalized
There are  892  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Fibroblast
Malignant
Monocyte
NK
Pl

Traceback (most recent call last):
  File "<ipython-input-12-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-12-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-17-8387889a0627>", line 9, in run_SingleCellNet
    dLevel=cell_type_key, stratify=True, limitToHVG=True, )
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/pySingleCellNet/scn_train.py", line 51, in scn_train
    sc.pp.highly_variable_genes(adNorm, min_mean=0.0125, max_mean=4, min_disp=0.5)
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 435, in highly_variable_genes
    flavor=flavor,
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 215, in _highly_variable_genes_single_batch
    df['mean_bin'] = pd.cut(df['means'], bin

snATACseq ccrcc_gene_activity (3000, 19843) (3000, 19843)
HVG
Matrix normalized
There are  772  classification genes

Endothelial
Epithelial
Fibroblast
Malignant
Monocyte
T cells
There are 594 top gene pairs

Finished pair transforming the data

snATACseq ccrcc_motif (3000, 633) (3000, 633)
HVG
None


Traceback (most recent call last):
  File "<ipython-input-12-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-12-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-17-8387889a0627>", line 9, in run_SingleCellNet
    dLevel=cell_type_key, stratify=True, limitToHVG=True, )
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/pySingleCellNet/scn_train.py", line 51, in scn_train
    sc.pp.highly_variable_genes(adNorm, min_mean=0.0125, max_mean=4, min_disp=0.5)
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 435, in highly_variable_genes
    flavor=flavor,
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 215, in _highly_variable_genes_single_batch
    df['mean_bin'] = pd.cut(df['means'], bin

snATACseq gbm_gene_activity (3389, 19891) (2876, 19891)
HVG
Matrix normalized
There are  968  classification genes

B cell
Endothelial
Fibroblast
Malignant
Microglia
Neuron
Oligodendrocytes
T cells
There are 790 top gene pairs

Finished pair transforming the data

snATACseq gbm_motif (3393, 633) (2872, 633)
HVG
None


Traceback (most recent call last):
  File "<ipython-input-12-ce2e432e20b0>", line 14, in run_workflow_for_datasets
    train, val, directory)
  File "<ipython-input-12-ce2e432e20b0>", line 64, in run_workflow
    df = workflow(train, val, CELL_TYPE_KEY)
  File "<ipython-input-17-8387889a0627>", line 9, in run_SingleCellNet
    dLevel=cell_type_key, stratify=True, limitToHVG=True, )
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/pySingleCellNet/scn_train.py", line 51, in scn_train
    sc.pp.highly_variable_genes(adNorm, min_mean=0.0125, max_mean=4, min_disp=0.5)
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 435, in highly_variable_genes
    flavor=flavor,
  File "/home/estorrs/miniconda3/envs/pollock_dev/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py", line 215, in _highly_variable_genes_single_batch
    df['mean_bin'] = pd.cut(df['means'], bin

snRNAseq brca (5252, 29175) (4893, 29175)
HVG
Matrix normalized
There are  1164  classification genes

Adipocyte
B cell
CD4 T cell
CD8 T cell
Dendritic
Endothelial
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1243 top gene pairs

Finished pair transforming the data

snRNAseq ccrcc (4754, 33538) (4518, 33538)
HVG
Matrix normalized
There are  1112  classification genes

CD4 T cell
CD8 T cell
Dendritic
Endothelial
Epithelial
Fibroblast
Malignant
Monocyte
NK
Plasma
Treg
There are 1058 top gene pairs

Finished pair transforming the data

snRNAseq gbm (3722, 29748) (3577, 29748)
HVG
Matrix normalized
There are  1003  classification genes

B cell
Endothelial
Fibroblast
Malignant
Microglia
Monocyte
Neuron
Oligodendrocytes
T cells
There are 890 top gene pairs

Finished pair transforming the data



In [26]:
run_workflow_for_cross_disease(adata_map, run_SingleCellNet, 'SingleCellNet', RESULTS_CROSS_DISEASE_DIR)

scRNAseq brca_train_brca_val (6080, 27131) (5773, 27131)
HVG
Matrix normalized
There are  1077  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Endothelial
Erythrocyte
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1263 top gene pairs

Finished pair transforming the data

scRNAseq brca_train_cesc_val (6080, 27131) (4277, 22928)
HVG
Matrix normalized
There are  1077  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Endothelial
Erythrocyte
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1263 top gene pairs

Finished pair transforming the data

scRNAseq brca_train_hnscc_val (6080, 27131) (5203, 26929)
HVG
Matrix normalized
There are  1077  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Endothelial
Erythrocyte
Fibroblast
Malignant
Mast
Monocyte
NK
Plasma
Treg
There are 1263 top gene pairs

Finished pair transforming the data

scRNAseq brca_train_melanoma_val (6080, 27131) (3532, 23452)
HVG
Matrix normalized
There are  1077  cl

There are 1068 top gene pairs

Finished pair transforming the data

scRNAseq myeloma_train_cesc_val (3617, 24020) (4277, 22928)
HVG
Matrix normalized
There are  943  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Erythrocyte
Malignant
Mast
Monocyte
NK
Plasma
Platlete
There are 1068 top gene pairs

Finished pair transforming the data

scRNAseq myeloma_train_hnscc_val (3617, 24020) (5203, 26929)
HVG
Matrix normalized
There are  943  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Erythrocyte
Malignant
Mast
Monocyte
NK
Plasma
Platlete
There are 1068 top gene pairs

Finished pair transforming the data

scRNAseq myeloma_train_melanoma_val (3617, 24020) (3532, 23452)
HVG
Matrix normalized
There are  943  classification genes

B cell
CD4 T cell
CD8 T cell
Dendritic
Erythrocyte
Malignant
Mast
Monocyte
NK
Plasma
Platlete
There are 1068 top gene pairs

Finished pair transforming the data

scRNAseq myeloma_train_myeloma_val (3617, 24020) (3312, 24020)
HVG
Matrix normali

There are 594 top gene pairs

Finished pair transforming the data

snATACseq ccrcc_gene_activity_train_brca_motif_val (3000, 19843) (3519, 633)
HVG
Matrix normalized
There are  782  classification genes

Endothelial
Epithelial
Fibroblast
Malignant
Monocyte
T cells
There are 594 top gene pairs

Finished pair transforming the data

snATACseq ccrcc_gene_activity_train_ccrcc_gene_activity_val (3000, 19843) (3000, 19843)
HVG
Matrix normalized
There are  782  classification genes

Endothelial
Epithelial
Fibroblast
Malignant
Monocyte
T cells
There are 594 top gene pairs

Finished pair transforming the data

snATACseq ccrcc_gene_activity_train_ccrcc_motif_val (3000, 19843) (3000, 633)
HVG
Matrix normalized
There are  782  classification genes

Endothelial
Epithelial
Fibroblast
Malignant
Monocyte
T cells
There are 594 top gene pairs

Finished pair transforming the data

snATACseq ccrcc_gene_activity_train_gbm_gene_activity_val (3000, 19843) (2875, 19891)
HVG
Matrix normalized
There are  782  cl

In [ ]:
# run_workflow_for_cross_datatype(adata_map, run_SingleCellNet, 'SingleCellNet', RESULTS_CROSS_DTYPE_DIR)

###### testing stuff

In [ ]:
train, val = (sc.read_h5ad(adata_map['snATACseq']['brca_motif']['train']),
              sc.read_h5ad(adata_map['snATACseq']['brca_motif']['val']))

In [ ]:
if 'sparse' not in str(type(train.X)): train.X = scipy.sparse.csr_matrix(train.X)
if 'sparse' in str(type(val.X)): val.X = val.X.toarray()

In [ ]:
train.var

In [ ]:
len(set(train.var.index))

In [ ]:
cgenesA, xpairs, tspRF = pySCN.scn_train(train,
            nTopGenes = 100, nRand = 100, nTrees = 1000 ,nTopGenePairs = 100,
            dLevel = "cell_type", stratify=True, limitToHVG=True, )

In [ ]:
train.shape, val.shape

In [ ]:
val.obs

In [ ]:
tspRF

In [ ]:
val.X

In [ ]:
val.X = val.X.toarray()

In [ ]:
predictions = pySCN.scn_classify(val, cgenesA, xpairs, tspRF, nrand = 0)


In [ ]:
predictions.obs

In [ ]:
df = pd.merge(predictions.obs[['SCN_class']], val.obs, left_index=True, right_index=True)

df = df[['cell_type', 'SCN_class']]
df.index.name = 'cell_id'
df.columns = ['groundtruth', 'predictions']
df['probability'] = [np.nan] * df.shape[0]
df

##### MARS

In [ ]:
from args_parser import get_parser
from model.mars import MARS
from model.experiment_dataset import ExperimentDataset
from sklearn.utils.linear_assignment_ import linear_assignment

In [ ]:
##pulled from evaluation.py and modified to remove class thing
def hungarian_match(y_true, y_pred):
    """Matches predicted labels to original using hungarian algorithm."""
    
    y_true = adjust_range(y_true)
    y_pred = adjust_range(y_pred)
    
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    # Confusion matrix.
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_assignment(-w)
    d = {i:j for i, j in ind}
    y_pred = np.array([d[v] for v in y_pred])
    
    return y_true, y_pred


def adjust_range(y):
    """Assures that the range of indices if from 0 to n-1."""
    y = np.array(y, dtype=np.int64)
    val_set = set(y)
    mapping = {val:i for  i,val in enumerate(val_set)}
    y = np.array([mapping[val] for val in y], dtype=np.int64)
    return y
    
    
def run_mars_workflow(train, val, cell_type_key):
    params, unknown = get_parser().parse_known_args()
    params.device = 'cpu'
    
    if 'sparse' not in str(type(train.X)): train.X = scipy.sparse.csr_matrix(train.X)
    if 'sparse' not in str(type(val.X)): val.X = scipy.sparse.csr_matrix(val.X)
        
    var_names = train.var_names.intersection(val.var_names)
    train = train[:, var_names]
    val = val[:, var_names]

    train.obs['dataset'] = 'train'
    val.obs['dataset'] = 'val'

    adata = anndata.concat((train, val))

    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.scale(adata, zero_center=True, max_value=10.)

    sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')
    sc.pp.pca(adata, n_comps=50)
    
    train = adata[adata.obs['dataset'] == 'train',:]
    val = adata[adata.obs['dataset'] == 'val',:]

    train = train[train.obs.sort_values(cell_type_key).index]
    val = val[val.obs.sort_values(cell_type_key).index]
    
    class_to_int = {c:i for i, c in enumerate(sorted(set(train.obs[cell_type_key])))}
    int_to_class = {i:c for c, i in class_to_int.items()}
    
    y_train = np.array([class_to_int[c] for c in train.obs[cell_type_key]], dtype=np.int64)
    annotated = ExperimentDataset(train.X.toarray(), train.obs_names, train.var_names, 'train', y_train, )
    y_val = np.array([class_to_int[c] for c in val.obs[cell_type_key]], dtype=np.int64)
    unannotated = ExperimentDataset(val.X.toarray(), val.obs_names, val.var_names, 'val', y_val)
    
    pretrain_data = ExperimentDataset(val.X.toarray(), val.obs_names, val.var_names, 'val')
    n_clusters = len(np.unique(unannotated.y))
    mars = MARS(n_clusters, params, [annotated], unannotated, pretrain_data, hid_dim_1=1000, hid_dim_2=100)
    a, landmarks, scores = mars.train(evaluation_mode=True, save_all_embeddings=True)
    
    preds = a[a.obs['experiment']=='val'].copy()
    preds.obs.index = [x.replace('-val', '') for x in preds.obs.index]
    _, adjusted = hungarian_match(preds.obs['truth_labels'], preds.obs['MARS_labels'])
    preds.obs['adjusted_predicted'] = adjusted
    preds.obs['groundtruth'] = [val.obs.loc[x, cell_type_key] for x in preds.obs.index]
    preds.obs['predicted'] = [int_to_class[x] for x in preds.obs['adjusted_predicted']]

    df = preds.obs[['groundtruth', 'predicted']]
    df['cell_id'] = df.index.to_list()
    df['probability'] = np.nan
    df = df[['cell_id', 'groundtruth', 'predicted', 'probability']]

    return df

In [ ]:
run_workflow_for_datasets(adata_map, run_mars_workflow, 'mars', RESULTS_DIR)

In [ ]:
run_workflow_for_cross_disease(adata_map, run_mars_workflow, 'mars', RESULTS_CROSS_DISEASE_DIR)

In [ ]:
# run_workflow_for_cross_datatype(adata_map, run_mars_workflow, 'mars', RESULTS_CROSS_DTYPE_DIR)

###### testing stuff

In [ ]:
from model.mars import MARS
from model.experiment_dataset import ExperimentDataset

In [ ]:
train, val = (sc.read_h5ad(adata_map['scRNAseq']['pbmc']['train']),
              sc.read_h5ad(adata_map['scRNAseq']['pbmc']['val']))

In [ ]:
if 'sparse' not in str(type(train.X)): train.X = scipy.sparse.csr_matrix(train.X)
if 'sparse' not in str(type(val.X)): val.X = scipy.sparse.csr_matrix(val.X)


In [ ]:
var_names = train.var_names.intersection(val.var_names)
train = train[:, var_names]
val = val[:, var_names]

train.obs['dataset'] = 'train'
val.obs['dataset'] = 'val'

combined = train.concat(val)

sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.scale(adata, zero_center=True)

sc.pp.neighbors(adata, n_neighbors=30, use_rep='X')
sc.pp.pca(adata, n_comps=50)

In [ ]:
train = adata[adata.obs['dataset'] == 'train',:]
val = adata[adata.obs['dataset'] == 'val',:]

In [ ]:
y_train = np.array(train.obs['cell_type'])
annotated = ExperimentDataset(train.X.toarray(), train.obs_names, train.var_names, 'train', y_train)

In [ ]:
y_val = np.array(val.obs['cell_type'])
unannotated = ExperimentDataset(val.X.toarray(), val.obs_names, val.var_names, 'val', y_val)

In [ ]:
pretrain_data = ExperimentDataset(val.X.toarray(), val.obs_names, val.var_names, 'val')

In [ ]:

n_clusters = len(np.unique(unannnotated.y))

In [ ]:
mars = MARS(n_clusters, params, [annotated], unannnotated, pretrain_data, hid_dim_1=1000, hid_dim_2=100)

In [ ]:
# return both annotated and unannotated datasets with save_all_embeddings
adata, landmarks, scores = mars.train(evaluation_mode=True, save_all_embeddings=True) # evaluation mode

In [ ]:
adata.obs